In [5]:
import pygame, sys, random
import numpy as np
import math
import matplotlib.pyplot as plt
import pylab
from RVO_Py_MAS.RVO import RVO_update, reach, compute_V_des
from RVO_Py_MAS.vis import visualize_traj_dynamic
import time

num_episodes = 250

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment

boundaryPos = [100, 100]
boundaryLength = [70,70]
boundaryRadius = 40
dispSize = [1280, 960]
initPosAgentStandard = [dispSize[0] - 100, 100]
initPosAgent = initPosAgentStandard
goalPos = [100, 900]
rangeNumber = 5
obsNumber = 100

In [6]:
ws_model = dict()
ws_model['robot_radius'] = agentRadius
ws_model['circular_obstacles'] = []
ws_model['boundary'] = []

In [7]:
def rangeFinder(allObsPos, rangeCenter):
    indexMin = []
    tmpDist = [0 for _ in range(obsNumber)]
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
        tmpDist[i] = allObsAgtDistance[i]
    tmpDist.sort()
    for i in range(0, rangeNumber):
        indexMin.append(allObsAgtDistance.index(tmpDist[i]))
    return indexMin

In [8]:
#### pygame.init()
screen = pygame.display.set_mode(dispSize)
screen.fill([220, 220, 220])

rList, episodes = [], []
fps = 0

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
# for i in range(0,obsNumber):
#     obstaclePos[i][0] = int(150 + i % 7 * 150)
#     obstaclePos[i][1] = int(150 + i / 7 * 150)
# Number of Obstacles = 100
for i in range(0,obsNumber):
    obstaclePos[i][0] = int(150 + i % 10 * 100)
    obstaclePos[i][1] = int(80 + i / 10 * 100)
# Number of Obstacles = 150
# Number of Obstacles = 200
# for i in range(0,obsNumber):
#     obstaclePos[i][0] = int(150 + i % 14 * 80)
#     obstaclePos[i][1] = int(80 + i / 14 * 80)

index = rangeFinder(obstaclePos, initPosAgent)
rangeObstacle = [[0,0] for _ in range(rangeNumber)]
for j in range(0, rangeNumber):
    rangeObstacle[j][0] = obstaclePos[index[j]][0]
    rangeObstacle[j][1] = obstaclePos[index[j]][1]

for e in range(num_episodes):
    # Initialize
#     start = time.time()
#     frame = 0
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    
    X = [initPosAgent] + rangeObstacle
    V = [[0,0] for i in xrange(len(X))]
    V_max = [2.0 for i in xrange(len(X))]
#     goal = [goalPos] + rangeObstacle
    randomNo = [0 for _ in range(rangeNumber)]
    goal = [goalPos]
    for i in range(0, rangeNumber):
        randomNo[i] = random.randrange(0, 100)
        goal += [initPosAgent]

    print("Episode ", e, "Starts!")

    while not done:
#         frame += 1
        V_des = compute_V_des(X, goal, V_max)
        V = RVO_update(X, V_des, V, ws_model)

        x += int(V[0][0])
        y += int(V[0][1])
        
        collisionFlag = 0
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
        
        if math.sqrt((x -  goalPos[0])**2 + (y - goalPos[1])**2) <= agentRadius * 2:
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        index = rangeFinder(obstaclePos, [x,y])
        for j in range(0, rangeNumber):
#             V[j + 1][0] = random.randrange(-1, 2)
#             V[j + 1][1] = random.randrange(-1, 2)
            rangeObstacle[j][0] = obstaclePos[index[j]][0] + int(round(V[j + 1][0]))
            rangeObstacle[j][1] = obstaclePos[index[j]][1] + int(round(V[j + 1][1]))
            obstaclePos[index[j]][0] = rangeObstacle[j][0]
            obstaclePos[index[j]][1] = rangeObstacle[j][1]
            
            if math.sqrt((x - rangeObstacle[j][0])**2 + (y - rangeObstacle[j][1])**2) < 19:
                print("Collision!")
                collisionFlag = -1
                done = True
        k = 0
        for i in range(0,obsNumber):
            if i == index[0] or i == index[1] or i == index[2] or i == index[3] or i == index[4]:
                pygame.draw.circle(screen, [255, 255, 50], rangeObstacle[k], obstacleRadius, 0)
                k += 1
            else:
                pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
        goal = [goalPos]
        for i in range(0, rangeNumber):
            goal += [[rangeObstacle[i][0] + random.randrange(-1,2), rangeObstacle[i][1] + random.randrange(-1,2)]]
#             if i == rangeNumber - 1:
#                 goal += [rangeObstacle[0]]
#             else:
#                 goal += [rangeObstacle[i + 1]]
        X = [[x, y]] + rangeObstacle
#         goal = [goalPos, obstaclePos[index[0]], obstaclePos[index[1]], obstaclePos[index[2]], obstaclePos[index[3]], obstaclePos[index[4]]]

        if done:
            if collisionFlag == 1:
                rList.append(1)
            elif collisionFlag == -1:
                rList.append(0)
            # every episode, plot the play time
            initPosAgent = initPosAgentStandard
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            # Number of Obstacles = 50
#             for i in range(0,obsNumber):
#                 obstaclePos[i][0] = int(150 + i % 7 * 150)
#                 obstaclePos[i][1] = int(150 + i / 7 * 150)
            # Number of Obstacles = 100
            for i in range(0,obsNumber):
                obstaclePos[i][0] = int(150 + i % 10 * 100)
                obstaclePos[i][1] = int(80 + i / 10 * 100)
            # Number of Obstacles = 150
#             for i in range(0,obsNumber):
#                 obstaclePos[i][0] = int(150 + i % 12 * 100)
#                 obstaclePos[i][1] = int(80 + i / 12 * 100)
            # Number of Obstacles = 200
#             for i in range(0,obsNumber):
#                 obstaclePos[i][0] = int(150 + i % 14 * 80)
#                 obstaclePos[i][1] = int(80 + i / 14 * 80)

            episodes.append(e)
            
        pygame.draw.circle(screen, [100,255,100], goalPos, 10, 0)
        pygame.display.flip()
        screen.fill([220,220,220])
#         final = time.time()
#         elapsed = final - start
#         if frame >= 10:
#             break
#     fps += frame / elapsed
#     print("Frame Per Second: " + str(frame / elapsed))
    
    print("Percent of successful episodes: " + str(100.0 * sum(rList)/(e + 1)) + "%")
print("fps average: " + str(fps / (e + 1)))

('Episode ', 0, 'Starts!')
Collision!
Percent of successful episodes: 0.0%
('Episode ', 1, 'Starts!')
Collision!
Percent of successful episodes: 0.0%
('Episode ', 2, 'Starts!')
Goal Reached!
Percent of successful episodes: 33.3333333333%
('Episode ', 3, 'Starts!')
Goal Reached!
Percent of successful episodes: 50.0%
('Episode ', 4, 'Starts!')
Collision!
Percent of successful episodes: 40.0%
('Episode ', 5, 'Starts!')
Collision!
Percent of successful episodes: 33.3333333333%
('Episode ', 6, 'Starts!')
Goal Reached!
Percent of successful episodes: 42.8571428571%
('Episode ', 7, 'Starts!')
Collision!
Percent of successful episodes: 37.5%
('Episode ', 8, 'Starts!')
Collision!
Percent of successful episodes: 33.3333333333%
('Episode ', 9, 'Starts!')
Collision!
Percent of successful episodes: 30.0%
('Episode ', 10, 'Starts!')
Collision!
Percent of successful episodes: 27.2727272727%
('Episode ', 11, 'Starts!')
Collision!
Percent of successful episodes: 25.0%
('Episode ', 12, 'Starts!')
Collis

Goal Reached!
Percent of successful episodes: 34.693877551%
('Episode ', 98, 'Starts!')
Collision!
Percent of successful episodes: 34.3434343434%
('Episode ', 99, 'Starts!')
Collision!
Percent of successful episodes: 34.0%
('Episode ', 100, 'Starts!')
Collision!
Percent of successful episodes: 33.6633663366%
('Episode ', 101, 'Starts!')
Goal Reached!
Percent of successful episodes: 34.3137254902%
('Episode ', 102, 'Starts!')
Goal Reached!
Percent of successful episodes: 34.9514563107%
('Episode ', 103, 'Starts!')
Goal Reached!
Percent of successful episodes: 35.5769230769%
('Episode ', 104, 'Starts!')
Goal Reached!
Percent of successful episodes: 36.1904761905%
('Episode ', 105, 'Starts!')
Collision!
Percent of successful episodes: 35.8490566038%
('Episode ', 106, 'Starts!')
Goal Reached!
Percent of successful episodes: 36.4485981308%
('Episode ', 107, 'Starts!')
Goal Reached!
Percent of successful episodes: 37.037037037%
('Episode ', 108, 'Starts!')
Collision!
Percent of successful ep

Collision!
Percent of successful episodes: 34.8958333333%
('Episode ', 192, 'Starts!')
Collision!
Percent of successful episodes: 34.7150259067%
('Episode ', 193, 'Starts!')
Collision!
Percent of successful episodes: 34.5360824742%
('Episode ', 194, 'Starts!')
Goal Reached!
Percent of successful episodes: 34.8717948718%
('Episode ', 195, 'Starts!')
Goal Reached!
Percent of successful episodes: 35.2040816327%
('Episode ', 196, 'Starts!')
Collision!
Percent of successful episodes: 35.0253807107%
('Episode ', 197, 'Starts!')
Collision!
Percent of successful episodes: 34.8484848485%
('Episode ', 198, 'Starts!')
Collision!
Percent of successful episodes: 34.6733668342%
('Episode ', 199, 'Starts!')
Collision!
Percent of successful episodes: 34.5%
('Episode ', 200, 'Starts!')
Collision!
Percent of successful episodes: 34.328358209%
('Episode ', 201, 'Starts!')
Collision!
Percent of successful episodes: 34.1584158416%
('Episode ', 202, 'Starts!')
Goal Reached!
Percent of successful episodes: 3

In [5]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()

Percent of successful episodes: 43.2%
